In [11]:
import librosa
import soundfile as sf
import noisereduce as nr
from scipy import signal
import numpy as np
import os
import whisper
from IPython.display import Audio


### 데이터셋 만들기

In [12]:
model = whisper.load_model("base")

In [13]:
result = model.transcribe("/Users/changhyeoncheon/dev/Vreeze-AI/Audio/hajin.m4a")

result["text"][:300]

/opt/anaconda3/envs/py3_11_9/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


' 다행이야. 그래요? 이 가격에 이런 품질이면 정말 실망스러워요. 좀 더 나은 서비스를 기대했는데 안돼. 요즘 어떻게 지내? 오랜만이라 반갑다. 지난번 만난 이유로 많은 일이 있었어. 정말 고마워요. 좋아. 이 책 정말 재밌어. 잘 지내? 뭐해요? 오늘 저녁은 제가 요리할게요. 뭐가 먹고 싶으세요? 제가 자신 있는 요리는 파스타에요. 알겠습니다. 진짜요? 좋은 아침. 어디 가세요? 잘 먹었어요. 내일 보자. 정말 실망이에요. 어제 뭐 했어? 미안해 약속에 늦어서 포스가 지연되었어. 오래 기다렸지? 지금 바빠요? 나 좀 도와줘. 어떻'

In [14]:
for r in result['segments']:
    print(f'[{r["start"]:.2f} --> {r["end"]:.2f}] {r["text"]}')

[0.00 --> 5.50]  다행이야. 그래요?
[5.50 --> 10.08]  이 가격에 이런 품질이면 정말 실망스러워요.
[10.08 --> 12.74]  좀 더 나은 서비스를 기대했는데
[12.74 --> 14.74]  안돼.
[14.74 --> 19.04]  요즘 어떻게 지내? 오랜만이라 반갑다.
[19.04 --> 22.14]  지난번 만난 이유로 많은 일이 있었어.
[22.14 --> 24.64]  정말 고마워요.
[24.64 --> 26.14]  좋아.
[26.74 --> 29.38]  이 책 정말 재밌어.
[29.38 --> 31.64]  잘 지내?
[31.64 --> 33.64]  뭐해요?
[33.64 --> 36.00]  오늘 저녁은 제가 요리할게요.
[36.00 --> 37.54]  뭐가 먹고 싶으세요?
[37.54 --> 40.94]  제가 자신 있는 요리는 파스타에요.
[40.94 --> 42.84]  알겠습니다.
[42.84 --> 44.64]  진짜요?
[44.64 --> 47.14]  좋은 아침.
[47.14 --> 49.34]  어디 가세요?
[49.34 --> 51.80]  잘 먹었어요.
[51.80 --> 53.80]  내일 보자.
[53.80 --> 56.40]  정말 실망이에요.
[56.40 --> 58.64]  어제 뭐 했어?
[58.64 --> 60.68]  미안해 약속에 늦어서
[60.68 --> 62.38]  포스가 지연되었어.
[62.38 --> 64.74]  오래 기다렸지?
[64.74 --> 66.84]  지금 바빠요?
[66.84 --> 69.10]  나 좀 도와줘.
[69.10 --> 71.50]  어떻게 지내세요?
[71.50 --> 74.00]  전화 좀 해줄래?
[74.00 --> 76.60]  와 대박이다.
[76.60 --> 78.94]  이거 얼마에요?
[78.94 --> 81.84]  별고파 밥 먹자.
[81.84 --> 84.46]  죄송합니다만 지금은 개인사정으로
[84.46 --> 87.

In [15]:
for i, r in enumerate(result['segments']):
    !ffmpeg -y -i /Users/changhyeoncheon/dev/Vreeze-AI/Audio/hajin.m4a -ss {r["start"]} -to {r["end"]} -hide_banner -loglevel error wavs/audio{i+1}.wav

In [16]:
Audio("wavs/audio1.wav")

In [17]:
with open("metadata.txt", "w", encoding="utf-8") as f:
    for i, r in enumerate(result['segments']):
        f.write(f"audio{i+1}|{r['text'].strip()}|{r['text'].strip()}\n")

### 프리프로세싱

In [18]:

def preprocess_voice(input_file, output_file, target_sr=24000, noise_reduction_strength=0.5):
    # 1. 오디오 로드
    audio, sr = librosa.load(input_file, sr=None)
    
    # 2. 노이즈 제거 (강도 조절)
    audio = nr.reduce_noise(
        y=audio, 
        sr=sr, 
        prop_decrease=noise_reduction_strength,  # 노이즈 감소 비율 조절
        stationary=False   # 비정적 노이즈 처리
    )
    
    # 3. 무음 제거
    audio, _ = librosa.effects.trim(audio, top_db=20)
    
    # 4. 볼륨 정규화
    audio = librosa.util.normalize(audio)
    
    # 5. 리샘플링
    if sr != target_sr:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
        sr = target_sr
    
    # 6. 주파수 필터링
    sos = signal.butter(10, [80, 8000], 'bandpass', fs=sr, output='sos')
    audio = signal.sosfilt(sos, audio)
    
    # 7. 결과 저장
    sf.write(output_file, audio, sr)
    
    return audio, sr

# 입력 및 출력 폴더 설정
input_folder = "wavs"
output_folder = "wavs_processed"

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# wavs 폴더의 모든 오디오 파일 처리
audio_files = [f for f in os.listdir(input_folder) if f.endswith(('.wav', '.mp3', '.m4a', '.flac', '.ogg'))]

for i, audio_file in enumerate(audio_files):
    input_path = os.path.join(input_folder, audio_file)
    
    # 확장자를 .wav로 통일
    output_filename = os.path.splitext(audio_file)[0] + ".wav"
    output_path = os.path.join(output_folder, output_filename)
    
    print(f"처리 중 ({i+1}/{len(audio_files)}): {audio_file}")
    
    try:
        preprocess_voice(
            input_path, 
            output_path,
            noise_reduction_strength=0.3  # 노이즈 제거 강도 낮춤
        )
        print(f"완료: {output_path}")
    except Exception as e:
        print(f"오류 발생 ({audio_file}): {str(e)}")

print(f"모든 파일 처리 완료. 총 {len(audio_files)}개 파일 중 성공적으로 처리된 파일: {sum(1 for f in os.listdir(output_folder) if f.endswith('.wav'))}")

처리 중 (1/176): audio87.wav
완료: wavs_processed/audio87.wav
처리 중 (2/176): audio93.wav
완료: wavs_processed/audio93.wav
처리 중 (3/176): audio78.wav
완료: wavs_processed/audio78.wav
처리 중 (4/176): audio44.wav
완료: wavs_processed/audio44.wav
처리 중 (5/176): audio50.wav
완료: wavs_processed/audio50.wav
처리 중 (6/176): audio151.wav
완료: wavs_processed/audio151.wav
처리 중 (7/176): audio145.wav
완료: wavs_processed/audio145.wav
처리 중 (8/176): audio144.wav
완료: wavs_processed/audio144.wav
처리 중 (9/176): audio150.wav
완료: wavs_processed/audio150.wav
처리 중 (10/176): audio51.wav
완료: wavs_processed/audio51.wav
처리 중 (11/176): audio45.wav
완료: wavs_processed/audio45.wav
처리 중 (12/176): audio79.wav
완료: wavs_processed/audio79.wav
처리 중 (13/176): audio92.wav
완료: wavs_processed/audio92.wav
처리 중 (14/176): audio86.wav
완료: wavs_processed/audio86.wav
처리 중 (15/176): audio90.wav
완료: wavs_processed/audio90.wav
처리 중 (16/176): audio84.wav
완료: wavs_processed/audio84.wav
처리 중 (17/176): audio53.wav
완료: wavs_processed/audio53.wav
처리 중 (18/176): 

#### (주의) 아래 코드는 수동으로 특정 오디오 파일의 길이를 자름

In [21]:
import librosa
import soundfile as sf
import numpy as np

def trim_audio_start(input_file, output_file, trim_seconds):
    """
    오디오 파일의 앞부분을 지정된 시간만큼 자르는 함수
    
    Parameters:
    input_file (str): 입력 오디오 파일 경로
    output_file (str): 출력 오디오 파일 경로
    trim_seconds (float): 앞에서 자를 시간(초)
    """
    # 오디오 파일 로드
    y, sr = librosa.load(input_file, sr=None)
    
    # 잘라낼 샘플 수 계산
    trim_samples = int(trim_seconds * sr)
    
    # 앞부분 자르기
    if trim_samples >= len(y):
        raise ValueError("자르려는 시간이 오디오 길이보다 깁니다.")
    
    trimmed_audio = y[trim_samples:]
    
    # 결과 저장
    sf.write(output_file, trimmed_audio, sr)
    
    print(f"앞에서 {trim_seconds}초 자른 파일이 {output_file}에 저장되었습니다.")
    print(f"원본 길이: {len(y)/sr:.2f}초, 잘린 후 길이: {len(trimmed_audio)/sr:.2f}초")
    
    return trimmed_audio, sr

# 사용 예시: wavs_processed/audio1.wav 파일의 앞에서 0.5초 자르기
input_file = "wavs_processed/audio1.wav"
output_file = "wavs_processed/audio1.wav"
trim_seconds = 2.4  # 앞에서 자를 시간(초), 필요에 따라 조정

# 함수 실행
trim_audio_start(input_file, output_file, trim_seconds)

앞에서 2.4초 자른 파일이 wavs_processed/audio1.wav에 저장되었습니다.
원본 길이: 4.95초, 잘린 후 길이: 2.55초


(array([ 0.00036621,  0.00012207,  0.00024414, ..., -0.01016235,
        -0.01077271, -0.01040649], dtype=float32),
 24000)

In [22]:
Audio("wavs_processed/audio1.wav")

### zip으로 압축

In [23]:
!zip -r text_split.zip ./wavs_processed ./metadata.txt
!cp -a ./text_split.zip ./text_split

updating: metadata.txt (deflated 74%)
  adding: wavs_processed/ (stored 0%)
  adding: wavs_processed/audio87.wav (deflated 7%)
  adding: wavs_processed/audio93.wav (deflated 6%)
  adding: wavs_processed/audio78.wav (deflated 12%)
  adding: wavs_processed/audio44.wav (deflated 16%)
  adding: wavs_processed/audio50.wav (deflated 6%)
  adding: wavs_processed/audio151.wav (deflated 7%)
  adding: wavs_processed/audio145.wav (deflated 8%)
  adding: wavs_processed/audio144.wav (deflated 8%)
  adding: wavs_processed/audio150.wav (deflated 8%)
  adding: wavs_processed/audio51.wav (deflated 8%)
  adding: wavs_processed/audio45.wav (deflated 10%)
  adding: wavs_processed/audio79.wav (deflated 7%)
  adding: wavs_processed/audio92.wav (deflated 5%)
  adding: wavs_processed/audio86.wav (deflated 10%)
  adding: wavs_processed/audio90.wav (deflated 7%)
  adding: wavs_processed/audio84.wav (deflated 12%)
  adding: wavs_processed/audio53.wav (deflated 8%)
  adding: wavs_processed/audio47.wav (deflated 9